In [ ]:
%pip install transformers
%pip install datasets
%pip install nltk scikit-learn --quiet
%pip install ray



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 MB 31.0 MB/s eta 0:00:00


In [ ]:
#  Import libraries
import os
import gc
import json
import numpy as np
import pandas as pd
import torch
import nltk
import sklearn
import string
import spacy



from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datasets import Dataset as Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import EarlyStoppingCallback
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    RobertaConfig,
    Trainer,
    TrainingArguments,
)

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# File Paths
train_file_path = "/content/drive/MyDrive/nlp_project/train.jsonlist"
validation_file_path = "/content/drive/MyDrive/nlp_project/validation.jsonlist"
test_file_path = "/content/drive/MyDrive/nlp_project/test.jsonlist"

# Helper function to process JSON into datasets
def json_to_dataset(path):
    dataset = []
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                l = json.loads(line.strip())
                if "verdict" in l and "body" in l:
                    newL = {
                        "labels": 1 if l["verdict"].lower() == "not the asshole" else 0,
                        "text": l["body"]
                    }
                    dataset.append(newL)
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Skipping line due to error: {e}")
                continue
    return Dataset.from_pandas(pd.DataFrame(dataset))

# Load datasets
dataset_train = json_to_dataset(train_file_path)
dataset_validation = json_to_dataset(validation_file_path)
dataset_test = json_to_dataset(test_file_path)

print(dataset_train)
print(dataset_validation)
print(dataset_test)


Dataset({
    features: ['labels', 'text'],
    num_rows: 32000
})
Dataset({
    features: ['labels', 'text'],
    num_rows: 4000
})
Dataset({
    features: ['labels', 'text'],
    num_rows: 4000
})


In [ ]:
# Load Tokenizer
print("building tokenizer...")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"], padding="max_length", truncation=True, max_length=512
    )
    tokenized["labels"] = examples["labels"]
    return tokenized

tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_validation = dataset_validation.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

print("First Tokenized Example:")
print(tokenized_dataset_train[0])

building tokenizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

First Tokenized Example:
{'labels': 1, 'text': "[TITLE] [AITA] Hockey Fan [BODY] I went to a hockey game at my college and since I didn't have season tickets to sit in the student section, got the nosebleed seats around center ice about 6 rows from the top.  It was against our rivals, so the seats were packed.  Every time the action came around the boards on our side, I had to stand up to see what was happening.  I also stood when we got close to scoring a goal.  The people behind me were pissed that I kept standing up and told me to just watch what was happening on the jumbotron.  My thoughts were that if I wanted to sit down and watch the game on a screen, I would have stayed at home and watched from my couch.  What are your thoughts; am I the asshole?", 'input_ids': [0, 10975, 47217, 3850, 742, 646, 250, 2068, 250, 742, 8471, 11232, 646, 387, 37588, 742, 38, 439, 7, 10, 5006, 177, 23, 127, 1564, 8, 187, 38, 399, 75, 33, 191, 3308, 7, 2662, 11, 5, 1294, 2810, 6, 300, 5, 8658, 5225, 1

In [ ]:
# Metrics computation
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    #predictions = logits.argmax(axis=-1)

    if isinstance(logits, tuple):
        logits = logits[0]
    predictions = np.argmax(logits, axis=-1)

    print("Labels:", labels)
    print("Predictions:", predictions)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="binary", zero_division=0
    )
    accuracy = accuracy_score(labels, predictions)

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
from ray import tune

# Model setup with BERT
model_config = RobertaConfig.from_pretrained("roberta-base", num_labels=2, hidden_dropout_prob=0.2, attention_probs_dropout_prob=0.2)
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config=model_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/nlp_project/model_output",
    num_train_epochs=3,              # Keep 3, starts overfitting around 4-5 epochs
    per_device_train_batch_size=16,  # Keep lower batch size = better metrics, slower
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    gradient_checkpointing=True,     # Enable gradient checkpointing for mem optimization
    fp16=True,                        # Enable Mixed precision training = faster, save mem
    logging_steps=50,                # TBD
    learning_rate=1e-5,              # Keep low = better metrics
    weight_decay=0.1,               # TBD:
    warmup_steps=500,                # TBD: Gradual learning rate warmup
    lr_scheduler_type="cosine",      # Keep cosine, slightly better than linear
    save_total_limit=1,              # Save only the best model
    load_best_model_at_end=True,     # Load the best model at the end
    gradient_accumulation_steps=4   # TBD: Accumulate gradients to simulate larger batch size
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

# Define a custom Trainer to use Cross-Entropy Loss explicitly
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):

        labels = inputs.pop("labels")  # Extract labels from inputs
        outputs = model(**inputs)  # Forward pass
        logits = outputs.logits  # Extract logits from the outputs

        # Use Cross-Entropy Loss
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_validation,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early stopping
)

# Train the model
trainer.train()


# Evaluate model on the test set
test_results = trainer.evaluate(tokenized_dataset_test)
print(f"Test set results: {test_results}")

# Save the model and tokenizer
model.save_pretrained("/content/drive/MyDrive/nlp_project/model_output/aita_roBERTa4_classifier_model")
tokenizer.save_pretrained("/content/drive/MyDrive/nlp_project/model_output/aita_roBERTa4_classifier_tokenizer")

env: CUDA_LAUNCH_BLOCKING=1


<ipython-input-58-104de59b884c>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.668000,0.662459,0.583250,0.509850,0.618844,0.433500
2,0.604900,0.654692,0.643500,0.627287,0.657174,0.600000


Labels: [1 1 1 ... 0 0 0]
Predictions: [1 0 1 ... 0 0 0]
Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 1 ... 0 1 0]


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.668000,0.662459,0.583250,0.509850,0.618844,0.433500
2,0.604900,0.654692,0.643500,0.627287,0.657174,0.600000
3,0.563900,0.654055,0.647250,0.639591,0.653786,0.626000


Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 1 ... 0 1 0]


Labels: [1 1 1 ... 0 0 0]
Predictions: [1 0 1 ... 1 0 0]
Test set results: {'eval_loss': 0.6759609580039978, 'eval_accuracy': 0.64525, 'eval_f1': 0.6297938951213149, 'eval_precision': 0.6584833606110202, 'eval_recall': 0.6035, 'eval_runtime': 9.5043, 'eval_samples_per_second': 420.864, 'eval_steps_per_second': 13.152, 'epoch': 3.0}


('/content/drive/MyDrive/nlp_project/model_output/aita_roBERTa4_classifier_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/nlp_project/model_output/aita_roBERTa4_classifier_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/nlp_project/model_output/aita_roBERTa4_classifier_tokenizer/vocab.json',
 '/content/drive/MyDrive/nlp_project/model_output/aita_roBERTa4_classifier_tokenizer/merges.txt',
 '/content/drive/MyDrive/nlp_project/model_output/aita_roBERTa4_classifier_tokenizer/added_tokens.json')

In [ ]:
# COMMON WORK EXTACTION

nlp = spacy.load("en_core_web_sm")

# Extend stop words to include overly common narrative terms
non_context_words = {"tell", "want", "say", "get", "go", "know", "think", "feel", "ask", "year","have","new","stay","place", "see", "happen","hour","see" }
for word in non_context_words:
    nlp.Defaults.stop_words.add(word)

# Function to extract refined context words
def extract_context_words(texts, top_n=12):
    context_words = []
    for doc in nlp.pipe(texts, batch_size=100, disable=["ner", "parser"]):
        # Filter tokens: exclude stop words, punctuation, and overly generic terms
        filtered_tokens = [
            token.lemma_.lower()
            for token in doc
            if not token.is_stop and
               token.is_alpha and
               token.pos_ in {"NOUN", "VERB", "ADJ"}
        ]
        context_words.extend(filtered_tokens)
    # Exclude extremely high-frequency terms common across both classes
    common_threshold = len(texts) * 0.3  # 30% occurrence threshold
    context_count = Counter(context_words)
    filtered_context = {k: v for k, v in context_count.items() if v < common_threshold}
    return Counter(filtered_context).most_common(top_n)

# Grouped data for analysis
def get_most_common_words_by_verdict(dataset, top_n=12):
    grouped_words = {"Not the Asshole": [], "Asshole": []}

    for example in dataset:
        verdict = "Not the Asshole" if example["labels"] == 1 else "Asshole"
        grouped_words[verdict].append(example["text"])

    # Process each group and extract top words
    results = {}
    for verdict, texts in grouped_words.items():
        common_words = extract_context_words(texts, top_n)
        results[verdict] = common_words

    return results

# Example usage
result = get_most_common_words_by_verdict(dataset_train, top_n=12)

# Display the results
for verdict, words in result.items():
    print(f"\n{verdict}:")
    for word, count in words:
        print(f"{word}: {count}")


In [ ]:
# NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE
# Calculate Average Number of Tokens in Training Dataset
# Check for potential nan values in specific columns of input data
for data in [tokenized_dataset_train, tokenized_dataset_validation, tokenized_dataset_test]:
    for column in ['input_ids', 'attention_mask', 'token_type_ids', 'labels']: # Check numerical columns
        nan_count = np.isnan(data[column]).sum()
        if nan_count > 0:
            print(f"NaN values found in column '{column}' of input data: {nan_count}")

def calculate_token_count(example):
    tokens = tokenizer(
        example["text"], truncation=False, padding=False
    )  # Ensure no truncation to get the full token count
    return len(tokens["input_ids"])

# Apply token count calculation to the training dataset
tokenized_dataset_train = tokenized_dataset_train.map(
    lambda example: {"token_count": calculate_token_count(example)}
)

# Calculate the average token count
average_tokens = tokenized_dataset_train["token_count"]
average_token_count = sum(average_tokens) / len(average_tokens)

print(f"Average number of tokens in the training dataset: {average_token_count:.2f}")
 #TESTING TESTING
print("Training Labels:", [example["labels"] for example in tokenized_dataset_train])
print("Validation Labels:", [example["labels"] for example in tokenized_dataset_validation])

print("Number of labels in model config:", model.config.num_labels)

print("Sample tokenized input:", tokenized_dataset_train[0])


 # Add sentiment and external features


def add_sentiment_features(dataset):
    sid = SentimentIntensityAnalyzer()

    def compute_sentiment(example):
        sentiment_scores = sid.polarity_scores(example["text"])
        example["sentiment"] = sentiment_scores["compound"]
        return example

    return dataset.map(compute_sentiment)

# Add sentiment features
dataset_train = add_sentiment_features(dataset_train)
dataset_validation = add_sentiment_features(dataset_validation)
dataset_test = add_sentiment_features(dataset_test)

# Normalize features
def normalize_features(dataset):
    scaler = MinMaxScaler()
    sentiment_scores = [example["sentiment"] for example in dataset]
    sentiment_scaled = scaler.fit_transform(np.array(sentiment_scores).reshape(-1, 1)).flatten()
    for i, example in enumerate(dataset):
        example["sentiment"] = sentiment_scaled[i]
    return dataset


dataset_train = normalize_features(dataset_train)
dataset_validation = normalize_features(dataset_validation)
dataset_test = normalize_features(dataset_test)